### ECE/CS/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2022 ###

# Optimal Power Line Upgrades #

#### Sofia Taylor (smtaylor8@wisc.edu)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
  1. [Background and Contribution](#1.1-Background-and-Contribution)
  1. [Data and Scenarios](#1.2-Data-and-Scenarios)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

### 1.1 Background and Contribution ###

Electric power lines can ignite wildfires when an electric fault occurs. Power line faults arise through multiple mechanisms, including contact with vegetation or animals, downed towers and lines, and equipment malfunctions. High-energy fault currents can lead to arcing and sparking, which can ignite a fire under hot and dry conditions.

There are several methods that are currently implemented to mitigate the wildfire ignition risk from power lines. On a shorter-term (operation) time horizon, it is possible to de-energize lines in high fire risk areas in what is known as a "public safety power shutoff". If there is no current in a power line, then there is no chance for a fault to occur. While these power shutoffs are effective in mitigating fire risk, they can result in power outages, which are particularly harmful for people that rely on electrically-powered medical devices and socially vulnerable populations.

On a longer-term (planning) time horizon, grid owners can conduct vegetation management, replace aging components, and convert overhead lines to underground cables. These upgrades, especially undergrounding lines, are expensive, so it is important to prioritize lines that exhibit the highest risk of igniting a fire. 

In this project, I propose an optimization-based method to choose lines for upgrade so that wildfire risk, upgrade costs, and load shed due to power shutoffs are minimized throughout an electric grid. This is a challenging problem because we must include constraints to represent how power flows throughout an electric grid. When considering multiple scenarios, the problem becomes large. Further, we must make a relaxation (via a McCormick envelope) in the power flow formulation to make the problem convex and linear. 

### 1.2 Data and Scenarios ###

The wildfire risk of each line in an electric grid is quantified using wildfire risk maps, specifically the Wildland Fire Potential Index ([WFPI](https://www.usgs.gov/fire-danger-forecast/wildland-fire-potential-index-wfpi?msclkid=6007b855d07511ec9923099beeaceb51)) from the US Geological Survey. The WFPI contains indices from 0 to 150 that represent the relative wildfire potential in an area due to natural (not power-specific) factors. The WFPI maps are published once daily, and the map for June 6, 2021 is shown in Fig. 1. 



<!-- ![WFPI](WFPI_map.png) -->

<div>
<img src="WFPI_map.png" width="500"/>
</div>

<div align="center"><b> Fig. 1 - Wildland Fire Potential Index (WFPI) map for June 6, 2021. </b></div>

This problem requires wildfire risk values for each line in an electric grid. The power system model that is used is the Reliability Test System from the Grid Modernization Lab Consortium ([RTS-GMLC](https://github.com/GridMod/RTS-GMLC?msclkid=41650ba3d07511ec8bb3deb3607abde1)), which is a synthetic grid model with an artificial location in the southwestern United States. Fig. 2 shows the RTS overlaid on a WFPI map.

<div>
<img src="RTS_map.png" width="500"/>
</div>

<div align="center"><b> Fig. 2 - Reliability Test System from the Grid Modernization Lab Consortium (RTS-GMLC). </b></div>

For this project, the risk of each power line is computed as the maximum intersecting WFPI value. The assignments are made using ArcGIS Pro, a mapping software application. For more information on this risk assignment, please refer to [this arXiv paper](https://arxiv.org/abs/2110.07348?msclkid=7ce57b87d07511ec8fe4c203cf031cfc). 

Since wildfire risk varies over time, it is important to consider multiple days (or scenarios) of wildfire risk. Therefore, the proposed model selects a set of lines to upgrade such that wildfire risk and load shed due to power shutoffs across multiple scenarios are minimized. In reality, load also varies over time, but the load is assumed to be constant for the purposes of this project, and load data can be added in future development. 

One challenge of considering multiple scenarios is that the size of the problem (i.e. number of variables and constraints) becomes large. To illustrate how this model works while still keeping the run time low, only XX scenarios are used in this example.

## 2. Mathematical model ##

As outlined below, the optimization problem has a linear objective function, linear constraints, bound constraints, and integrality constraints, making it a mixed integer linear program (MILP).
A McCormick envelope has been used to relax the original mixed-integer nonlinear program (MINLP) so that it becomes convex and linear. The $w_{l,k}$ variable in the first constraint actually represents (and replaced) the nonlinear function $z^{upgrade}_l\cdot z^{on}_{l,k}$. Thus, the solution obtained from solving this problem will be a lower bound solution on the original problem.

### 2.1 Parameters
The parameters are:

> $\boldsymbol{c_l}$ cost of upgrading line $l$

> $\boldsymbol{D_{l,k}}$ demand for each load $d$ in scenario $k$

> $\boldsymbol{R_{l,k}}$ wildfire risk values for each line $l$ in scenario $k$

> $\boldsymbol{\alpha}, \boldsymbol{\beta}, \boldsymbol{\gamma}$  tradeoff parameters for cost, risk, and load shed, respectively

> $\boldsymbol{b}_{l,i,j}$ series susceptance of branch $l$

> $\boldsymbol{\theta}^\mathrm{max}$ a positive number with magnitude greater than the maximum possible angle difference on any branch

> $\boldsymbol{\theta}^\mathrm{min}$ a negative number with magnitude greater than the maximum possible angle difference on any branch

> $\boldsymbol{T}_{l}$ active power thermal limit on branch $l$

> $\overline{\boldsymbol{P}_{g}}$ maximum active power output of generator $g$

> $\underline{\boldsymbol{P}_{g}}$ minimum active power output of generator $g$

### 2.2 Decision variables
The decision variables are:

> $z^{upgrade}_l$   a binary decision for whether line $l$ is upgraded. Equals 0 if line is not upgraded, and 1 if line is upgraded.

> $z^{on}_{l,k}$  a binary decision for whether line $l$ is on in scenario $k$. Equals 0 if line is turned off, and 1 if line is on.

> $w_{l,k}$ a McCormick Envelope constraint for line $l$  in scenario $k$, equal to the product ($z^{upgrade}_l \cdot z^{on}_{l,k}$)

> $\rho_k$ system-wide wildfire risk in scenario $k$

> $\delta_k$ system-wide load shed in scenario $k$

> $\theta_{i,k}$ voltage angle of bus $i$ in scenario $k$

> $P^G_{g,k}$  active power generation for generator $g$ in scenario $k$

> $p_{l,k}^L$ active power flow on line $l$ in scenario $k$
    

### 2.3 Objective

The objective is to minimize upgrade cost, wildfire risk, and load shed due to power shutoffs.

$$\min\quad \displaystyle\boldsymbol{\alpha}\cdot\sum_{\mathcal{l\in\mathcal{L}^\mathrm{upgrade}}}\boldsymbol{c}_{l}\cdot z_{l}^\mathrm{upgrade} + \sum_{k\in\mathcal{K}} \rho_k + \sum_{k\in\mathcal{K}} \delta_k $$

The upgrade cost term is zero if the line is not upgraded ($z_{l}^\mathrm{upgrade} = 0$) and non-zero if the line is upgraded ($z_{l}^\mathrm{upgrade} = 1$). The values of $\rho_k$ and $\delta_k$ are defined by the constraints below.

### 2.4 Constraints

To define the wildfire risk $\rho_k$ in each scenario $k$, we need to use the risk parameters $\boldsymbol{R}_{l,k}$ in addition to the upgrade and shutoff decision variables, $z^{upgrade}_l$ and $z^{on}_{l,k}$, respectively. We want to describe three cases:

1. If the line is upgraded (i.e. it is an underground line), then the wildfire risk is zero, regardless of the shutoff status.

2. If the line is not upgraded (i.e. it is an overhead line) and the line is turned off, then the wildfire risk is zero.

3. If the line is not upgraded and the line is on, then the wildfire risk is a non-zero value equal to the assigned WFPI value.

Mathematically, we can write these expressions as the following constraint:

\begin{align*}
    & \rho_k\geq \boldsymbol{\beta}\cdot\sum_{l\in\mathcal{L}}\boldsymbol{R}_{l,k} \cdot z_{l,k}^\mathrm{on}(1- z^{upgrade}_l) &\forall k\in\mathcal{K}.\\
\end{align*}

This is equivalent to

\begin{align*}
    & \rho_k\geq\boldsymbol{\beta}\cdot\sum_{l\in\mathcal{L}}\boldsymbol{R}_{l,k} \cdot z_{l,k}^\mathrm{on}-\boldsymbol{R}_{l,k}\cdot (z_{l,k}^\mathrm{on}\cdot z^{upgrade}_l) &\forall k\in\mathcal{K}.\\
\end{align*}

However, the above constraint, the product of $z^{upgrade}_l$ and $z^{on}_{l,k}$ is nonlinear, so we replace that expression with a McCormick envelope variable $ w_{l,k}$ and constraints. 
In the implemented model, the system-wide wildfire $\rho_k$ risk and load shed $\delta_k$ for each scenario $k$ are defined as

\begin{align*}
    & \rho_k\geq\boldsymbol{\beta}\cdot\sum_{l\in\mathcal{L}}\boldsymbol{R}_{l,k} \cdot z_{l,k}^\mathrm{on}-\boldsymbol{R}_{l,k}\cdot  w_{l,k} &\forall k\in\mathcal{K}\\
	& \delta_k\geq - \boldsymbol{\gamma}\cdot\sum_{d\in D}\boldsymbol{D}_{d,k}\cdot x_{d,k}^\mathrm{load} &\forall k\in\mathcal{K}
\end{align*}

where $z_{l}^\mathrm{upgrade}$ and $z_{l,k}^\mathrm{on}$ are binary and $x_{d,k}^\mathrm{load}$ is a continuous variable between 0 and 1:

\begin{align*}
	& z_{l}^\mathrm{upgrade}\in\{0,1\} &\forall l\in\mathcal{L}\\
	&z_{l,k}^\mathrm{on}\in\{0,1\}&\forall l\in\mathcal{L},k\in\mathcal{K}\\
	&0\le x_{d,k}^\mathrm{load}\le 1\quad &\forall d\in\mathcal{D},k\in\mathcal{K}.
\end{align*}

The McCormick envelope constraints, which represent the upper and lower bounds on $w_{l,k}$, are

\begin{align*}
	& w_{l,k} \geq 0 &\forall k\in\mathcal{K}\\
	& w_{l,k} \geq z_{l}^\mathrm{upgrade} + z_{l,k}^\mathrm{on} - 1 &\forall k\in\mathcal{K}\\
	& w_{l,k} \leq z_{l,k}^\mathrm{on} &\forall k\in\mathcal{K}\\
	& w_{l,k} \leq z_{l}^\mathrm{upgrade} &\forall k\in\mathcal{K}.
\end{align*}

The next set of constraints model how power flows throughout an electric grid. The power flow constraints include branch flow constraints, thermal limits, power balance equations, and active power limits. First, the branch flow constraints relate the power flow $p_{l,i,j,k}^L$ on a line $l$ between buses $i$ and $j$ to the line's susceptance $\boldsymbol{b}_{l,i,j}$, the voltage angles at buses $i$ and $j$. The terms $\boldsymbol{\theta}^\mathrm{max}$ and $\boldsymbol{\theta}^\mathrm{min}$ make it so the branch flow constraints are not enforced if the line is turned off.

\begin{align*}
	& p_{l,i,j,k}^L \le -\boldsymbol{b}_{l,i,j} (\theta_{i,k} - \theta_{j,k} + \boldsymbol{\theta}^\mathrm{max}(1-z_{l,k}^\mathrm{on})) & \forall l,i,j\in\mathcal{L},k\in\mathcal{K}  \\
	& p_{l,i,j,k}^L \ge -\boldsymbol{b}_{l,i,j} (\theta_{i,k} - \theta_{j,k} + \boldsymbol{\theta}^\mathrm{min} (1-z_{l,k}^\mathrm{on})) & \forall l,i,j\in\mathcal{L},k\in\mathcal{K}\\
\end{align*}

The thermal limits constrain the power flow on a power line based on the thermal characteristics of the line.

\begin{align*}
    &-\boldsymbol{T}_{l}z_{l,k}^\mathrm{on} \le p_{l,k}^L \le \boldsymbol{T}_{l}z_{l,k}^\mathrm{on} & \forall l\in\mathcal{L},k\in\mathcal{K} \\
\end{align*}


There is a power balance constraint for each bus (or node) so that the power generated, the branch power flow, and the load consumed at that bus sum to zero. Note that the load $\boldsymbol{D}_{d,k}$ is multiplied by the continuous variable $x_{d,k}^\mathrm{load}$, which takes on a value between zero and one, to represent how much of the load is actually delivered.

\begin{align*}
	&\sum_{g\in\mathcal{G}_i}p_{g,k}^G -
    \sum_{l,i,j\in\mathcal{L}_i^o}p_{l,i,j,k}^L -
    \sum_{d\in\mathcal{D}_i} \boldsymbol{D}_{d,k}\cdot x_{d,k}^\mathrm{load} = 0 \quad &\forall i \in \mathcal{B},k\in\mathcal{K}\\
\end{align*}


Next, the active power limits specify the upper and lower limits of the generator setpoints. In this formulation, the lower limits are set to zero, so that a generator can be completely turned off if necessary. 

\begin{align*}
	&\quad \underline{\boldsymbol{P}_{g}} \le p_{g,k}^G \le \overline{\boldsymbol{P}_g} \quad & \forall g \in \mathcal{G},k\in\mathcal{K}
\end{align*}


## 3. Solution ##

The following cells implement and solve the optimization model presented above. The code is split into sections for readability.

In it's current implementation, the number of scenarios $K$ is set to 10. In solving this model, I've found that solving for more than 10 scenarios takes a very long time on my personal computer. As a reminder, the risk values are changing across each scenario due to variability in the wildfire risk maps. The load demanded is not changing, although more accurate load data could be incorporated in the future.

In [41]:
# Load Julia Packages
#--------------------
# Uncomment and add packages if needed

# import Pkg
# Pkg.add("PowerModels")
# Pkg.add("PowerModelsWildfire")
# Pkg.add("Gurobi")
# Pkg.add("JuMP")
# Pkg.add("CSV")
# Pkg.add("DataFrames")

In [42]:
# Load Julia Packages
#--------------------
using Pkg; Pkg.activate(".")
using PowerModels, PowerModelsWildfire
using Gurobi, JuMP
using CSV, DataFrames

  Activating project at `C:\Users\smtaylor8\Documents\ECE 524\FinalProject_SofiaTaylor`


In [43]:
# Load and Configure Risk Data
# ----------------------------

# Read in branch risk values
branch_risk = CSV.read("RTSGMLC_Max_NoSgmt_20210101_20211231.csv", DataFrame)

# Get number of scenarios and first risk column from data
K = 0
for i = 1:length(names(branch_risk))
    if occursin("WFPI", names(branch_risk)[i])
        global K += 1
        if K == 1
            global riskColStart = i
        end
    end
end

# Uncomment and modify this line to manually change the total number of scenarios K that are run
# If commented, code will run for all 360 scenarios in csv
K = 10 

# Make transformer risk zero
for i = 1:size(branch_risk)[1]
    if branch_risk.Length[i] == 0 
        for k = 0:(K-1)
            branch_risk[i,riskColStart + k] = 0
        end
    end
end

# Create risk dictionary
R = Dict(branch_risk.OID_[i] =>collect(branch_risk[i,riskColStart:(riskColStart+K-1)]) for i in 1:size(branch_risk)[1])

# # Create cost dictionary
cost = Dict(branch_risk.OID_[i] => branch_risk.Length[i,:][1]*2000000 for i in 1:size(branch_risk)[1])

println("Risk and cost data is configured.")

Risk and cost data is configured.


In [44]:
# Load Power System Data
# ----------------------

# Load Matpower file of grid data
powermodels_path = joinpath(dirname(pathof(PowerModelsWildfire)), "..")
file_name = "$(powermodels_path)/test/networks/RTS_GMLC_risk.m"

# Store the power system data in the PowerModel dictionary format
data = PowerModels.parse_file(file_name)

# Add zeros to turn linear objective functions into quadratic ones
# so that additional parameter checks are not required
PowerModels.standardize_cost_terms!(data, order=2)

# Adds reasonable rate_a values to branches without them
PowerModels.calc_thermal_limits!(data)

# Note: ref contains all the relevant system parameters needed to build the OPS model
# When we introduce constraints and variable bounds below, we use the parameters in ref.
ref = PowerModels.build_ref(data) # use build_ref to filter out inactive components
ref_add_on_off_va_bounds!(ref, data)
ref = ref[:it][:pm][:nw][0]

println("Power system data is configured.")

[info | PowerModels]: extending matpower format with data: bus_risk 73x3
[info | PowerModels]: extending matpower format with data: gen_name 158x4
[info | PowerModels]: extending matpower format with constant data: risk_weight
[info | PowerModels]: extending matpower format with data: areas 3x3
[info | PowerModels]: extending matpower format with data: branch_risk 120x3
[info | PowerModels]: extending matpower format by appending matrix "branch_risk" in to "branch"
[info | PowerModels]: extending matpower format by appending matrix "gen_name" in to "gen"
[info | PowerModels]: extending matpower format by appending matrix "bus_risk" in to "bus"
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 32 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 32 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in

[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 111 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 63 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 63 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 115 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 115 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 92 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 92 from 180.0 to 60.0 de

[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 34 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 44 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 44 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 94 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 94 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 55 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 55 from 180.0 to 60.0 deg.


[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 57 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 8 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 8 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 64 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 64 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 19 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 19 from 180.0 to 60.0 deg.
[w

[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 83 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 45 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 45 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 68 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 68 from 180.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 56 from -180.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 56 from 180.0 to 60.0 deg.


[warn | PowerModels]: the voltage setpoint on generator 154 does not match the value at bus 309
[warn | PowerModels]: the voltage setpoint on generator 149 does not match the value at bus 118
[warn | PowerModels]: the voltage setpoint on generator 49 does not match the value at bus 301
[warn | PowerModels]: the voltage setpoint on generator 59 does not match the value at bus 315
[warn | PowerModels]: the voltage setpoint on generator 5 does not match the value at bus 102
[warn | PowerModels]: the voltage setpoint on generator 31 does not match the value at bus 207
[warn | PowerModels]: the voltage setpoint on generator 62 does not match the value at bus 315
[warn | PowerModels]: the voltage setpoint on generator 122 does not match the value at bus 119
[warn | PowerModels]: the voltage setpoint on generator 143 does not match the value at bus 118
[warn | PowerModels]: the voltage setpoint on generator 112 does not match the value at bus 215
[warn | PowerModels]: the voltage setpoint on 

[warn | PowerModels]: the voltage setpoint on generator 85 does not match the value at bus 215
[warn | PowerModels]: the voltage setpoint on generator 147 does not match the value at bus 118
[warn | PowerModels]: the voltage setpoint on generator 135 does not match the value at bus 313
[warn | PowerModels]: the voltage setpoint on generator 48 does not match the value at bus 301
[warn | PowerModels]: the voltage setpoint on generator 103 does not match the value at bus 313
[warn | PowerModels]: the voltage setpoint on generator 30 does not match the value at bus 202
[warn | PowerModels]: the voltage setpoint on generator 3 does not match the value at bus 101
[warn | PowerModels]: simplifying pwl cost on generator 78, [0.0, 0.0, 0.1666667, 0.0, 0.33333329999999994, 0.0, 0.5, 0.0] -> [0.0, 0.0, 0.5, 0.0]
[warn | PowerModels]: simplifying pwl cost on generator 81, [0.0, 0.0, 0.1666667, 0.0, 0.33333329999999994, 0.0, 0.5, 0.0] -> [0.0, 0.0, 0.5, 0.0]
[warn | PowerModels]: simplifying pwl c

[warn | PowerModels]: simplifying pwl cost on generator 125, [0.0, 0.0, 0.2103333, 0.0, 0.4206667, 0.0, 0.631, 0.0] -> [0.0, 0.0, 0.631, 0.0]
[warn | PowerModels]: simplifying pwl cost on generator 98, [0.0, 0.0, 0.172, 0.0, 0.344, 0.0, 0.516, 0.0] -> [0.0, 0.0, 0.516, 0.0]
[warn | PowerModels]: simplifying pwl cost on generator 113, [0.0, 0.0, 0.0853333, 0.0, 0.17066669999999998, 0.0, 0.256, 0.0] -> [0.0, 0.0, 0.256, 0.0]
[warn | PowerModels]: simplifying pwl cost on generator 110, [0.0, 0.0, 0.312, 0.0, 0.624, 0.0, 0.9359999999999999, 0.0] -> [0.0, 0.0, 0.9359999999999999, 0.0]
[warn | PowerModels]: simplifying pwl cost on generator 127, [0.0, 0.0, 0.2233333, 0.0, 0.4466667, 0.0, 0.67, 0.0] -> [0.0, 0.0, 0.67, 0.0]
[warn | PowerModels]: simplifying pwl cost on generator 157, [0.0, 0.0, 2.3783333, 0.0, 4.7566667, 0.0, 7.135, 0.0] -> [0.0, 0.0, 7.135, 0.0]
[warn | PowerModels]: simplifying pwl cost on generator 96, [0.0, 0.0, 0.1666667, 0.0, 0.33333329999999994, 0.0, 0.5, 0.0] -> [0.0,

In [45]:
# Initialize a JuMP Optimization Model
#-------------------------------------
model = Model(Gurobi.Optimizer)

# Add Variables
# -------------

# Add binary variables for the on/off status of all components in scenario k
@variable(model, z_on[(l,i,j) in ref[:arcs_from], k in 1:K], Bin)

# Add binary variables for the upgrade status of all branches
@variable(model, z_upgrade[(l,i,j) in ref[:arcs_from]], Bin)

# Add continous variables to indicate load served in scenario k
@variable(model, 0.0 <= x_load[l in keys(ref[:load]), k in 1:K] <= 1.0)

# Add variable for the total risk in scenario k
@variable(model, rho[k in 1:K] >= 0)

# Add variable for the total load shed in scenario k
@variable(model, delta[k in 1:K])

# Add voltage angles va for each bus i in scenario k
@variable(model, va[i in keys(ref[:bus]), k in 1:K])

# Add active power generation variable pg for each generator in scenario k
@variable(model, pg[i in keys(ref[:gen]), k in 1:K])

# Add power flow variables p to represent the active power flow for each branch in scenario k
@variable(model,  p[(l,i,j) in ref[:arcs_from], k in 1:K])

# Build JuMP expressions for the value of p[(l,i,j), k] and p[(l,j,i), k] on the branches
# note: this is used to make the definition of nodal power balance simpler
# A separate expression is needed for each scenario k
p_expr = Array{Dict{Tuple{Int64, Int64, Int64}, AffExpr}}(undef, 360)
for k = 1:K
    p_expr[k] = Dict([((l,i,j), 1.0*p[(l,i,j),k]) for (l,i,j) in ref[:arcs_from]])
    p_expr[k] = merge(p_expr[k], Dict([((l,j,i), -1.0*p[(l,i,j),k]) for (l,i,j) in ref[:arcs_from]]))
end

# Add variables for McCormick Envelope relaxation
# One variable is needed for each line and each scenario
@variable(model, w[(l,i,j) in ref[:arcs_from], k in 1:K] >= 0)

println("\nVariables added to model.")

Academic license - for non-commercial use only - expires 2022-06-04

Variables added to model.


In [46]:
# Add Objective Function
# ----------------------

# Set weighting factors
alpha = 1   # Cost weight
beta = 1    # Risk weight
gamma = 1  # Load weight

# Objective: minimize upgrade cost + risk + load shed
JuMP.@objective(model, Min,
     alpha*(sum(cost[l]*z_upgrade[(l,i,j)] for (l,i,j) in ref[:arcs_from])) # upgrade cost
     + sum(rho)         # wildfire risk
     + sum(delta)       # load shed due to power shutoffs
)

println("Objective function added to model.")

Objective function added to model.


In [52]:
alpha*(sum(cost[l]*value.(z_upgrade[(l,i,j)]) for (l,i,j) in ref[:arcs_from]))

1.88e9

In [57]:
value.(rho)

10-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [47]:
# Add Constraints
# ---------------

# Constraint to limit risk
for k in 1:K
    JuMP.@constraint(model, rho[k] >=
        beta*(sum(R[l][k]*z_on[(l,i,j), k] - R[l][k]*w[(l,i,j),k] for (l,i,j) in ref[:arcs_from])*sum(cost[l] for (l,i,j) in ref[:arcs_from])) 
    )
end

# Constraint to limit load shed
for k in 1:K
    JuMP.@constraint(model, delta[k] >=
        - gamma*(sum(x_load[i,k]*load["pd"] for (i,load) in ref[:load])*sum(cost[l] for (l,i,j) in ref[:arcs_from]))/100
    )
end

# Fix the voltage angle to zero at the reference bus
for k in 1:K
    for (i,bus) in ref[:ref_buses]
        @constraint(model, va[i,k] == 0)
    end
end

# Branch constraints
for k in 1:K
    for (i,branch) in ref[:branch]

        # Build the from variable id of the i-th branch, which is a tuple given by (branch id, from bus, to bus)
        f_idx = (i, branch["f_bus"], branch["t_bus"])
        p_fr = p[f_idx, k]                     # p_fr is a reference to the optimization variable p[f_idx]
        z_br = z_on[f_idx, k]
        va_fr = va[branch["f_bus"],k]         # va_fr is a reference to the optimization variable va on the from side of the branch
        va_to = va[branch["t_bus"],k]         # va_fr is a reference to the optimization variable va on the to side of the branch
        # Compute the branch parameters and transformer ratios from the data
        g, b = PowerModels.calc_branch_y(branch)

        # Voltage angle difference limit
        JuMP.@constraint(model, va_fr - va_to <= branch["angmax"]*z_br + ref[:off_angmax]*(1-z_br))
        JuMP.@constraint(model, va_fr - va_to >= branch["angmin"]*z_br + ref[:off_angmin]*(1-z_br))

        # DC Power Flow Constraint
        if b <= 0
            JuMP.@constraint(model, p_fr <= -b*(va_fr - va_to + ref[:off_angmax]*(1-z_br)) )
            JuMP.@constraint(model, p_fr >= -b*(va_fr - va_to + ref[:off_angmin]*(1-z_br)) )
        else # account for bound reversal when b is positive
            JuMP.@constraint(model, p_fr >= -b*(va_fr - va_to + ref[:off_angmax]*(1-z_br)) )
            JuMP.@constraint(model, p_fr <= -b*(va_fr - va_to + ref[:off_angmin]*(1-z_br)) )
        end

        # Thermal limit
        JuMP.@constraint(model, p_fr <=  branch["rate_a"]*z_br)
        JuMP.@constraint(model, p_fr >= -branch["rate_a"]*z_br)

    end
end

# Nodal power balance constraints
for k in 1:K
    for (i,bus) in ref[:bus]

        # Build a list of the loads and shunt elements connected to the bus i
        bus_loads = [(l,ref[:load][l]) for l in ref[:bus_loads][i]]

        # Active power balance at node i
        @constraint(model,
            sum(pg[g,k] for g in ref[:bus_gens][i]) -                         # sum of active power generation at bus i -
            sum(p_expr[k][a] for a in ref[:bus_arcs][i]) -                    # sum of active power flow on lines from bus i -
            sum(x_load[l,k]*load["pd"] for (l,load) in bus_loads)            # sum of active load * load shed  at bus i = 0
            == 0
        )
    end
end

# Generator constraints
for k in 1:K
    for (i,gen) in ref[:gen]

        # Power limit
        JuMP.@constraint(model, pg[i,k] <= gen["pmax"])#*z_gen[i])
        JuMP.@constraint(model, pg[i,k] >= 0)#*z_gen[i])

    end
end

# McCormick Envelope upper and lower bound constraints
for k in 1:K
    for (i,branch) in ref[:branch]

        f_idx = (i, branch["f_bus"], branch["t_bus"])
        ω = w[f_idx,k]
        zup = z_upgrade[f_idx]
        zon = z_on[f_idx,k]

        @constraint(model, ω >= zup + zon - 1 )
        @constraint(model, ω <= zon)
        @constraint(model, ω <= zup)

    end
end

println("Constraints added to model.")

Constraints added to model.


In [48]:
# Solve the optimization problem
# ------------------------------

optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 13480 rows, 5940 columns and 37970 nonzeros
Model fingerprint: 0x9cabe80f
Variable types: 4620 continuous, 1320 integer (1320 binary)
Coefficient statistics:
  Matrix range     [7e-01, 1e+12]
  Objective range  [1e+00, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 8e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 0.0000000
Presolve removed 1980 rows and 720 columns
Presolve time: 0.15s
Presolved: 11500 rows, 5220 columns, 34620 nonzeros
Variable types: 2690 continuous, 2530 integer (2520 binary)

Root relaxation: objective -5.568402e+10, 7326 iterations, 0.48 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

## 4. Results and discussion ##

First, let's check that the solver terminated without an error.

<!-- | Quantity      | System Totals | Result Totals  |
| ------------- |:-------------:| --------------:|
| Risk          | 7142          | 730            |
| Load          | 85.50         |  44.62         |
| Cost          | 6.64e9        |   3.7e8        | -->


In [49]:
# Check that the solver terminated without an error
println("The solver termination status is $(termination_status(model))")

The solver termination status is OPTIMAL


Next, let's examine some of the relevant variable values.

In [50]:
# Print results
# -----------------------

# Check objective value, although the value does not have significance
println("The objective value is $(objective_value(model))")

# Compute load served in each scenario, based on load curtailment variables
for k in 1:K
    served_load = sum(value(x_load[i,k])*load["pd"] for (i,load) in ref[:load])
    total_load = sum(load["pd"] for (i,load) in ref[:load])
    println("The load served in scenario $k is $served_load, out of the total $total_load.")
end

# Compute total wildfire risk (prior to any upgrade or shutoff decisions)
total_risk = 0
for k in 1:K
    total_risk += sum(R[l][k] for (l,i,j) in ref[:arcs_from])
end

# Compute risk in each scenario and print results
for k in 1:K
    new_risk = sum(value.(z_on[(l,i,j),k])*R[l][k] for (l,i,j) in ref[:arcs_from])
    println("The system risk is $new_risk, reduced from $total_risk.")
end

# Compute and print the cost of upgrading lines
upgrade_cost = sum(cost[l]*value(z_upgrade[(l,i,j)]) for (l,i,j) in ref[:arcs_from])
total_cost = sum(cost[l] for (l,i,j) in ref[:arcs_from])
println("The upgrade cost is $upgrade_cost, out of the total cost of upgrading all lines $total_cost")

The objective value is -5.46928e10
The load served in scenario 1 is 85.19999999999999, out of the total 85.49999999999999.
The load served in scenario 2 is 85.19999999999999, out of the total 85.49999999999999.
The load served in scenario 3 is 85.19999999999999, out of the total 85.49999999999999.
The load served in scenario 4 is 85.19999999999999, out of the total 85.49999999999999.
The load served in scenario 5 is 85.19999999999999, out of the total 85.49999999999999.
The load served in scenario 6 is 85.19999999999999, out of the total 85.49999999999999.
The load served in scenario 7 is 85.19999999999999, out of the total 85.49999999999999.
The load served in scenario 8 is 85.19999999999999, out of the total 85.49999999999999.
The load served in scenario 9 is 85.19999999999999, out of the total 85.49999999999999.
The load served in scenario 10 is 85.19999999999999, out of the total 85.49999999999999.
The system risk is 2720.0, reduced from 71230.
The system risk is 2386.0, reduced fr

Finally, we can print the list of lines that are selected for upgrade.

In [59]:
# Get upgraded devices
for l in sort(collect(keys(ref[:branch])))
    f_idx = (l, ref[:branch][l]["f_bus"], ref[:branch][l]["t_bus"])
    value(z_upgrade[f_idx])==1 ? println("Branch $l is upgraded.") : false
end

Branch 3 is upgraded.
Branch 8 is upgraded.
Branch 11 is upgraded.
Branch 20 is upgraded.
Branch 21 is upgraded.
Branch 28 is upgraded.
Branch 29 is upgraded.
Branch 34 is upgraded.
Branch 40 is upgraded.
Branch 42 is upgraded.
Branch 44 is upgraded.
Branch 45 is upgraded.
Branch 51 is upgraded.
Branch 53 is upgraded.
Branch 59 is upgraded.
Branch 62 is upgraded.
Branch 64 is upgraded.
Branch 65 is upgraded.
Branch 68 is upgraded.
Branch 69 is upgraded.
Branch 71 is upgraded.
Branch 74 is upgraded.
Branch 76 is upgraded.
Branch 78 is upgraded.
Branch 82 is upgraded.
Branch 87 is upgraded.
Branch 91 is upgraded.
Branch 97 is upgraded.
Branch 101 is upgraded.
Branch 105 is upgraded.
Branch 106 is upgraded.
Branch 113 is upgraded.
Branch 116 is upgraded.


We can plot the lines selected for upgrade in ArcGIS Pro, as in Fig. 3.


<div>
<img src="upgrades.png" width="500"/>
</div>

<div align="center"><b> Fig. 3 - Power lines selected for upgrade in teal. </b></div>

The model successfully chooses a set of lines for upgrade that seems to be reasonable. We can see that many lines are selected for upgrade in this case, but we could reduce this by scaling the terms in the objective function to either more heavily weight upgrade costs or less heavily weight wildfire risk. The results are very sensitive to the choice of tradeoff parameters. In order to use this model to make grid planning decisions, one might want to sweep the tradeoff parameter values to examine which parameters (and thus which solution points) result in favorable levels of the three objects. The creation of such a tradeoff plot is a task that will be completed in future work. 

It is also possible to reformulate the problem so that one or more of the terms is limited by some minimum threshold. For example, we could specify a budget (or max cost) that can be spent on line upgrades, rather than simply minimizing cost. Alternatively, we could set a threshold for the maximum load that can be shed due to power shutoffs. These reformulations are also an area of future work. 

It is also important to note that the solution, though optimal, may not be entirely accurate due to the McCormick relaxation that is made.

## 5. Conclusion ##

The proposed model selects power lines to upgrade to reduce wildfire ignition risk, load shed due to public safety power shutoffs, and the cost of line upgrade investments. One strength of this model is that it utilitizes wildfire potential maps to estimate the potential for a power line to start a fire. To capture variability in wildfire potential, the model selects upgrades that are optimal across multiple scenarios. 

One limitation of this work is that the risk values obtained from the WFPI maps only represent the wildfire potential of the line's location, and do not represent the likelihood of an electric fault. Instead, we assume this probability is constant across the system. Another limitation is that load variability is not captured in this example, so incorporating this load data is an area for future work.

Another key challenge with this work is that it does not seem to scale well. In this report, only 10 scenarios are used in the solution because greater numbers result in a very long run time. This is likely due to the large number of variables and constraints that are needed for each scenario. Another area of future work is to try to reduce the problem size. One option for this that is currently in development is to utilize clustering to generate a small set of representative wildfire risk scenarios.

It would be interesting to extend this problem in several directions. One option is to consider the uncertainty in the wildfire risk values through stochastic optimization. Another direction is to incorporate the potential for other grid assets (such as distributed energy resources and microgrid capabilities) to reduce wildfire risk and load shed due to power shutoffs. 

## 6. Author Contributions

Sofia is the sole member of this project team, but Noah, David, and Prof. Line are given credit here for their roles in developing the model presented here.

#### 1. Modelling  
Sofia Taylor: %20  

Noah Rhodes: %20

David Sehloff: %20

Line Roald: %40
  
#### 2. Analysis  
Sofia Taylor: %100  

#### 3. Data Gathering  
Sofia Taylor: %100 

#### 4. Software Implementation  
Sofia Taylor: %100  

#### 5. Report writing and poster presentation   
Sofia Taylor: 100%